<a href="https://colab.research.google.com/github/bbanzai88/Data-Science-Repository/blob/main/Assemble_the_data_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This note book assemnbles the data. First we extract the FCS data, extract the CAS numbers for that. Then we manipulate thge FCS data and add one CAS number per line. Next we process the CEDI data. We use webchemto clean  up the CAS numbers in the CEDI data as they are malformed i.e. they dont have a _separating the different parts. Next, we obtain pubchem Id, GHS Codes and Precautionary Statements from pubchem. We then obtain more chemical information from  pubchem. Next we get information on each compound from  pubmed. We then assemble  the data..

In [ ]:
%reload_ext rpy2.ipython

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta
import re

In [19]:
# OFAS Interested Data Source 1: Inventory of Effective Food Contact Substance (FCS) Notifications
# URL: https://www.cfsanappsexternal.fda.gov/scripts/fdcc/?set=FCN

# STEP 1: Get the data for FDA approved FCS
FCS_inv_url = "https://www.cfsanappsexternal.fda.gov/scripts/fdcc/index.cfm?set=FCN&sort=FCN_No&order=DESC&showAll=true&type=basic&search=";
FCS_HTML_tables = pd.read_html(FCS_inv_url)

# STEP 2: Extract information from HTML Tables (i.e., scrapping of information)
FCS_table_data = FCS_HTML_tables[0]
FCN_df = pd.DataFrame (FCS_table_data, columns = ['FCN No.', 'Food Contact Substance','Manufacturer/Supplier','Effective Date'])

len(FCN_df)
# STEP 3: Extract Relevant information from scrapped data
CAS = FCN_df['Food Contact Substance'].str.extract(r'([0-9]+[\u2011|-][0-9]{2}[\u2011|-][0-9](?![0-9]))')
CAS.columns =['CAS']
CAS=CAS.drop_duplicates()
CAS.head(5)

,CAS
0,115-77-5
1,9003-04-7
2,29437-40-9
3,79-21-0
4,9010-77-9


In [ ]:
!pip install html-table-parser-python3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Library for opening url and creating
# requests
import urllib.request
 
# pretty-print python data structures
from pprint import pprint
 
# for parsing all the tables present
# on the website
from html_table_parser.parser import HTMLTableParser
 
# for converting the parsed data in a
# pandas dataframe
import pandas as pd
 
 
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):
 
    # Opens a website and read its
    # binary contents (HTTP Response Body)
 
    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
 
    #reading contents of the website
    return f.read()
 
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.cfsanappsexternal.fda.gov/scripts/fdcc/index.cfm?set=FCN&sort=FCN_No&order=DESC&showAll=true&type=basic&search="').decode('utf-8')
 
# Defining the HTMLTableParser object
p = HTMLTableParser()
 
# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)
 
# Now finally obtaining the data of
# the table required
#pprint(p.tables[0])
 
# converting the parsed data to
# dataframe
#print("\n\nPANDAS DATAFRAME\n")
df=pd.DataFrame(p.tables[1])


In [ ]:
df.shape

(1, 4)

In [ ]:
CAS = df[1].str.extract(r'([0-9]+[\u2011|-][0-9]{2}[\u2011|-][0-9](?![0-9]))')

In [ ]:
CAS.shape

(1, 1)

In [20]:
import requests
import pandas as pd

url = 'https://www.cfsanappsexternal.fda.gov/scripts/fdcc/index.cfm?set=FCN&sort=FCN_No&order=DESC&showAll=true&type=basic&search=0'
html = requests.get(url).content
df_list = pd.read_html(html)
df1 = df_list[0]

In [ ]:
df1

,FCN No. (sorted Z-A),Food Contact Substance,Manufacturer/Supplier,Effective Date
0,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022"
1,2228,"Cross-linked 2-propenoic acid, homopolymer, so...",Novipax,"Jul 14, 2022"
2,2226,Poly(ethyl acrylate-co-glycidyl methacrylate-c...,"PPG Industries, Inc.","Jul 6, 2022"
3,2224,An aqueous mixture containing peroxyacetic aci...,AFCO,"Jun 28, 2022"
4,2223,A reactive mixture of ethylene acrylic acid co...,Akzo Nobel Coatings Inc.,"Jun 23, 2022"
...,...,...,...,...
1586,5,Polybutadiene - graft - poly(methyl acrylate- ...,INEOS USA LLC,"Mar 14, 2000"
1587,4,Isophthalic acid or dimethyl isophthalate,INEOS Joliet LLC,"Mar 15, 2000"
1588,3,Styrene-acrylic copolymers produced by polymer...,Rohm and Haas Company,"Mar 9, 2000"
1589,2,"GENOXTM EP, chemically identified as Amines, b...",Addivant,"Mar 9, 2000"


In [ ]:
df1.columns

Index(['FCN No.  (sorted Z-A)', 'Food Contact Substance',
       'Manufacturer/Supplier', 'Effective Date'],
      dtype='object')

In [21]:
CAS = df1['Food Contact Substance'].str.extract(r'([0-9]+[\u2011|-][0-9]{2}[\u2011|-][0-9](?![0-9]))')

In [22]:
CAS

,0
0,115-77-5
1,9003-04-7
2,29437-40-9
3,79-21-0
4,9010-77-9
...,...
1586,27012-62-0
1587,NaN
1588,NaN
1589,204933-93-7


In [23]:
CAS2=CAS.drop_duplicates()
CAS2.columns = ['CAS']
CAS2['CAS'].astype(str)
#CAS2['CAS'] = CAS2['CAS'].str.replace('-','')
CAS2

,CAS
0,115-77-5
1,9003-04-7
2,29437-40-9
3,79-21-0
4,9010-77-9
...,...
1560,518-47-8
1561,137898-98-7
1568,250639-69-1
1572,62258-49-5


In [24]:
df1['dummy']=True
CAS2['dummy']=True
df2 = pd.merge(df1,CAS2, on='dummy')
 
df2.drop('dummy', axis=1, inplace=True)
 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import textdistance

In [33]:
#df2['Ratio'] = df2.apply(lambda x:fuzz.ratio(df2['CAS'],df2['Food Contact Substance']), axis=1)
df2['Partial_Ratio'] = df2[['CAS','Food Contact Substance']].apply(lambda x:fuzz.partial_ratio('df2.CAS','df.Food Contact Substance'), axis=1)
df2['Token_Set_Ratio'] = df2[['CAS','Food Contact Substance']].apply(lambda x:fuzz.token_set_ratio('df2.CAS','df.Food Contact Substance'), axis=1)
df['Token_Sort_Ratio'] = df2[['CAS','Food Contact Substance']].apply(lambda x:fuzz.token_sort_ratio('df2.CAS','df.Food Contact Substance'), axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [43]:
df2['Rank_Token_Set_Ratio'] = df2.groupby('CAS')['Token_Set_Ratio'].rank(ascending=False, method='dense')
FCS_Final = (df2['Rank_Token_Set_Ratio']== 1) & (df2['Token_Set_Ratio']>70)
FCS_Final.to_csv('/content/sample_data/FCS_Final.csv')

In [40]:
df2

,FCN No. (sorted Z-A),Food Contact Substance,Manufacturer/Supplier,Effective Date,CAS,Partial_Ratio,Token_Set_Ratio,Rank_Token_Set_Ratio
0,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022",115-77-5,43,31,1.0
1,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022",9003-04-7,43,31,1.0
2,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022",29437-40-9,43,31,1.0
3,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022",79-21-0,43,31,1.0
4,2230,Pentaerythritol (CAS Reg. No. 115-77-5). REPLA...,"Meredian, Inc.","Jul 29, 2022",9010-77-9,43,31,1.0
...,...,...,...,...,...,...,...,...
1121650,1,"Silver sodium hydrogen zirconium phosphate, rh...","Toagosei Company, Ltd.","Mar 1, 2000",518-47-8,43,31,1.0
1121651,1,"Silver sodium hydrogen zirconium phosphate, rh...","Toagosei Company, Ltd.","Mar 1, 2000",137898-98-7,43,31,1.0
1121652,1,"Silver sodium hydrogen zirconium phosphate, rh...","Toagosei Company, Ltd.","Mar 1, 2000",250639-69-1,43,31,1.0
1121653,1,"Silver sodium hydrogen zirconium phosphate, rh...","Toagosei Company, Ltd.","Mar 1, 2000",62258-49-5,43,31,1.0


In [ ]:
import requests
import pandas as pd

url = ' https://www.accessdata.fda.gov/scripts/sda/sdNavigation.cfm?sd=edisrev&displayAll=true';

html = requests.get(url).content
df_list2 = pd.read_html(html)
cedi = df_list2[0]
cedi = cedi.dropna(subset = ['CAS/ID NO']).copy()
cedi['CAS/ID NO'] = cedi['CAS/ID NO'].astype('int64')
cedi

,MAINTERM,CAS/ID NO,CUM DC (ppb),CEDI (mg/kg bw/d),REGNUM
0,"(1,1,4,4- TETRAMETHYLTETRAMETHYLENE)BIS(TERT-B...",78637,0.2,0.000010,177.2600 177.1520
1,"(2,4,4-TRIMETHYLPENT-2-YL)-N-PHENYL-1-NAPHTHYL...",68259369,50.0,0.002500,NaN
2,(2- (METHACRYLOYLOXY)ETHYL)TRIMETHYLAMMONIUM M...,6891447,0.4,0.000020,178.3520 176.170
3,(2-ALKENYL(C15-21))SUCCINIC ANHYDRIDE,68784123,5.0,0.000250,176.170
4,"(N-OCTYL)TIN S,S'S"" TRIS(ISOOCTYLMERCAPTOACETATE)",26401865,7.7,0.000385,178.2650
...,...,...,...,...,...
1297,ZINC NAPHTHENATE-DEHYDROABIETYLAMINE MIXTURE,977170936,7.0,0.000350,175.105
1298,ZINC NITRATE,7779886,7.0,0.000350,175.105
1299,ZINC PYRITHIONE,13463417,2.0,0.000100,NaN
1300,ZIRCONIUM BIS(ACETATE-O)OXO-,5153242,1.8,0.000090,NaN


In [ ]:
cedi.to_csv('/content/sample_data/cedi.csv')
cedi.columns

Index(['MAINTERM', 'CAS/ID NO', 'CUM DC (ppb)', 'CEDI (mg/kg bw/d)', 'REGNUM'], dtype='object')

In [ ]:
%%R

cedi <- read.csv("/content/sample_data/cedi.csv")
colnames(cedi)

[1] "X"                 "MAINTERM"          "CAS.ID.NO"        
[4] "CUM.DC..ppb."      "CEDI..mg.kg.bw.d." "REGNUM"           


In [ ]:
%%R

install.packages("webchem")
library(webchem)

cedi <- read.csv("/content/sample_data/cedi.csv")

cedi5=cedi

for (j in 1:dim(cedi)[1]) {
  q<-cedi5[j,3]
  print(q)
  Sys.sleep(5)
  casn=as.cas(q)
  print(casn)
  cedi5$CASN[j]=casn
}

write.csv(cedi5,"/content/sample_data/cedi5.csv", row.names = FALSE)

(as ‘lib’ is unspecified)














	‘/tmp/RtmpzloHcB/downloaded_packages’



[1] 78637
  78-63-7 
"78-63-7" 
[1] 68259369
  68259-36-9 
"68259-36-9" 
[1] 6891447
  6891-44-7 
"6891-44-7" 
[1] 68784123
  68784-12-3 
"68784-12-3" 
[1] 26401865
  26401-86-5 
"26401-86-5" 
[1] 32760808
  32760-80-8 
"32760-80-8" 
[1] 4118165
  4118-16-5 
"4118-16-5" 
[1] 94031
  94-03-1 
"94-03-1" 
[1] 75376
  75-37-6 
"75-37-6" 
[1] 646253
  646-25-3 
"646-25-3" 
[1] 64253301
  64253-30-1 
"64253-30-1" 
[1] 882073430
  882073-43-0 
"882073-43-0" 
[1] 2855278
  2855-27-8 
"2855-27-8" 
[1] 32687788
  32687-78-8 
"32687-78-8" 
[1] 3785340
  3785-34-0 
"3785-34-0" 
[1] 16068374
  16068-37-4 
"16068-37-4" 
[1] 491589221
  491589-22-1 
"491589-22-1" 
[1] 292648
  292-64-8 
"292-64-8" 
[1] 35691657
  35691-65-7 
"35691-65-7" 
[1] 18908162
  18908-16-2 
"18908-16-2" 
[1] 113669582
  113669-58-2 
"113669-58-2" 
[1] 106990436
  106990-43-6 
"106990-43-6" 
[1] 7779273
  7779-27-3 
"7779-27-3" 
[1] 26175702
  26175-70-2 
"26175-70-2" 
[1] 281867
  281-86-7 
"281-86-7" 
[1] 108452
  108-45-2 


In [ ]:

cedi5=pd.read_csv('/content/sample_data/cedi5.csv')
CAS5=cedi5['CASN']
CAS4=pd.concat([CAS5,CAS2])
del CAS4['CAS']
CAS4.columns = ['CAS']
CAS4.to_csv('/content/sample_data/df_inventory.csv')

CAS4.head()

,CAS
0,78-63-7
1,68259-36-9
2,6891-44-7
3,68784-12-3
4,26401-86-5



###
Now we have a list of CAS numbers. It is time to obtain information from pubchem  for each. To start with we will obtain pubchem Id, GHS Codes and Precautionary Statements from pubchem. 

In [ ]:
!pip install thermo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#  Most of these libraries are included in the Anaconda distribution of Python.  Download necessary libraries through the Anaconda prompt with pip install or conda install.  

import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from thermo.chemical import Chemical
import tkinter
from tkinter import *
from tkinter import filedialog
import os

In [ ]:
#  this cell opens a dialog to load the template spreadsheet

df_inventory = pd.read_csv('/content/sample_data/df_inventory.csv')

In [ ]:
#  this cell creates a temporary reference object for p codes and precautionary statements

result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/ghs/#_prec','lxml')
soup = BeautifulSoup(result.text,'lxml')

gross_precautions_list = list()
p_codes_list = list()
precaution_statements_list = list()

for i in soup.select('#pcode')[0].select('td'):
    gross_precautions_list.append(i.text)
for i in range(0,len(gross_precautions_list)):
    if not re.search(r'P\d\d\d',gross_precautions_list[i])==None:
        p_codes_list.append(gross_precautions_list[i])
for code in p_codes_list:
    precaution_statements_list.append(gross_precautions_list[gross_precautions_list.index(code)+1])
precaution_data_dict = {'P Codes':p_codes_list,'Precautionary Statements':precaution_statements_list}
df_precaution = pd.DataFrame(precaution_data_dict)

In [ ]:
#  Note: The loaded spreadsheet must contain the CAS numbers of the chemicals in a column named "CAS"
#  This cell creates new columns in the dataframe and cross-references PubChem IDs with given CAS numbers

df_inventory['PubChem ID']=np.nan
df_inventory['GHS Codes']=np.nan
df_inventory['Precautionary Statements']=np.nan
for i in df_inventory['CAS']:
    try:
        chem = Chemical(f'{i}')
        df_inventory.loc[df_inventory['CAS']==i,'PubChem ID'] = chem.PubChem
    except Exception:
        pass

In [ ]:
#  This cell will loop over the PubChem IDs and request GHS data from NCBI.  This could have a runtime of several minutes depending on your hardware and internet connection.

for chem_id in set(df_inventory['PubChem ID'].dropna()):
    result = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/{int(chem_id)}/JSON/?response_type=display&heading=GHS%20Classification','lxml')
    soup = BeautifulSoup(result.text,'lxml').text
    if len(soup) > 90:
        pattern_hits = [m.start() for m in re.finditer(r'"H\d\d\d', soup)]
        ghs_codes_set = set()
        for i in range(0,len(pattern_hits)):
            phrase_start = pattern_hits[i]
            phrase_end = soup[phrase_start+1:].find('"')
            ghs_codes_set.add(soup[phrase_start+1:phrase_start+5])
        ghs_codes_list = list()
        for c in ghs_codes_set:
            ghs_start = soup.find(c)
            ghs_end = soup[soup.find(c)+1:].find('"')+1
            ghs_codes_list.append(soup[ghs_start:ghs_start+ghs_end])
        joined_ghs = ' --- '.join(ghs_codes_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'GHS Codes'] = joined_ghs
        p_list = list()
        for i in set(soup[soup.find('Precautionary Statement Codes'):].replace('and ','').replace(' ','').replace('"\n}','').split(',')):
            if i in p_codes_list:
                p_list.append(i+' '+df_precaution['Precautionary Statements'][df_precaution[df_precaution['P Codes']==i].index[0]])
        joined_p = ' --- '.join(p_list)
        df_inventory.loc[df_inventory['PubChem ID']==chem_id,'Precautionary Statements'] = joined_p
    else:
        pass
df_inventory.head(100)

,Unnamed: 0,CAS,PubChem ID,GHS Codes,Precautionary Statements
0,0,78-63-7,6545.0,H242 (77.09%): Heating may cause a fire [Dange...,P403 Store in a well-ventilated place. --- P32...
1,1,68259-36-9,NaN,NaN,NaN
2,2,6891-44-7,NaN,NaN,NaN
3,3,68784-12-3,7922.0,H319 (80.31%): Causes serious eye irritation [...,P363 Wash contaminated clothing before reuse. ...
4,4,26401-86-5,NaN,NaN,NaN
...,...,...,...,...,...
95,95,7786-17-6,24582.0,H226 (100%): Flammable liquid and vapor [Warni...,P370+P378 In case of fire: Use ... to extingui...
96,96,61167-58-6,109058.0,,
97,97,14020-52-1,518858.0,NaN,NaN
98,98,77-62-3,6486.0,H413 (98.68%): May cause long lasting harmful ...,P501 Dispose of contents/container to ...


In [ ]:
df_inventory.to_csv('/content/sample_data/df_inventory_GHS.csv')

In [ ]:
#OFAS Interested Data Source 3: Pubchem Chemical Information


# URL: https://pubchem.ncbi.nlm.nih.gov/
#NOTE:Running this takes 1/2 hour
# STEP 1: #Below we retrieve SMILES Codes from the pubchem API based on the CAS # we extracted from Food Contact Substance
#! python3 - Script1.py - Retrieve SMILES codes from PubChem API


#see https://kellerbits.net/wordpress/?p=326


'''This script enables automatically connecting to the PubChem database, 
transfer of CAS numbers which are converted to CID identifiers
as first step and then resolved to respective SMILES codes.'''

# Import the library necessary for making a web service request.
from os import chdir
import urllib.request, urllib.error
import json
import time
import pandas as pd

# Define working directory
chdir('/content/sample_data/')

# Function for resolving given CAS number into CID. Therefore
# variables for PUG-REST request URL pieces are defined 

def cas_to_cid(cas):
    path_prolog = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug';
    path_compound = '/compound/'
    path_name = 'name/'
    path_cas = cas
    path_cas_rest = '/cids/JSON'

    url = path_prolog + path_compound + path_name + path_cas + path_cas_rest


# Make a PUG-REST request and store the output in "request"
    print('cas_to_cid:', url)
    try:
        request = urllib.request.urlopen(url)
    except urllib.error.HTTPError:
        print('HTTPError while requesting cas', cas)
        return ''

    # Give the output/reply back as JSON and return CID number from function
    if request is not None:
        reply = request.read()
        if reply is not None and len(reply) > 0:
            json_out = json.loads(reply)
            cid = json_out['IdentifierList']['CID'][0]
            return cid
    return ''

# Function for searching and extracting SMILES code with entering CID 

def cid_to_smiles(cid):
    path_prolog = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug';
    path_compound = '/compound/'
    path_name = 'cid/'
    path_cid = str(cid)
    path_cid_rest = '/property/CanonicalSMILES/JSON'

    url = path_prolog + path_compound + path_name + path_cid + path_cid_rest

# Make a PUG-REST request and store the output in "request"
    print('cid_to_smiles:', url)
    try:
        request = urllib.request.urlopen(url)
    except urllib.error.HTTPError:
        print('HTTPError while requesting cid', cid)
        return ''

    # Give the reply in JSON format, access and return the SMILES code
    if request is not None:
        reply = request.read()
        if reply is not None and len(reply) > 0:
            json_out = json.loads(reply)
            #return json_out
            smiles = json_out['PropertyTable']['Properties'][0]['CanonicalSMILES']
            return smiles
    return ''

# NOTE: to do this for many CAS numbers, iterate thru the given list and call above functions to 
# resolve to cid and, in turn, to SMILES. Sleep between each request to avoid overloading
# PubChem servers.

# Load list with CAS numbers where SMILES code is to be requested
df = pd.read_csv('/content/sample_data/df_inventory_GHS.csv', sep = ',')
df=df['CAS']
list_cas = df.astype(str).values.tolist()
list_cas=list_cas

# Both functions described above are now called in the third function map_cas_list_to_csv

def map_cas_list_to_csv(list_cas):
    output = ''
    for cas in list_cas:
        cid = cas_to_cid(cas)
        if len(str(cid)) > 0:
            smiles = cid_to_smiles(cid)
            if len(smiles) > 0:
                line = cas + '|' + str(cid) + '|' + smiles
                output = output + line + '\n' # create and concatenate output
                print(line)
                time.sleep(0.9) # sleep after each loop for 0,8 seconds
    return output

s_out = 'CAS|CID|SMILES\n'
output = map_cas_list_to_csv(list_cas) # call function for generating final result
final = s_out + output # now final contains a complete csv as string, just write it out to a file.


with open("RESULT_Substances_with_CAS_with_SMILES.csv", "w") as file:
    file.write(final)

# STEP 2: This gave us a file with CAS and SMILES
#RESULT_Substances_with_CAS_with_SMILES.csv
# STEP 3: Extract Relevant information from each SMILE

!pip install pubchempy
import pubchempy as pcp
import pandas as pd

df = pd.read_csv('/content/sample_data/RESULT_Substances_with_CAS_with_SMILES.csv', sep="|", engine='python')
df=df.drop_duplicates()

data = []

for i in df['SMILES']:
  time.sleep(10)
  try:
    props = pcp.get_properties(['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount'], i, 'smiles')

    data.append(props)
  except KeyError as e:
    continue

#data
len(data[0][0].keys())
rows = []
columns = data[0][0].keys()



from tqdm import tqdm


for i in tqdm(len(list_cas)):
#for i in range(20):
    rows.append(data[i][0].values())

props_df = pd.DataFrame(data=rows, columns=columns) 
props_df.insert(1, 'SMILES', df['SMILES'], True)
props_df.head()
props_df.to_csv('FCN Data with Properties.csv', index=False)




#[TODO]: None





Streaming output truncated to the last 5000 lines.
cas_to_cid: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/991-84-4/cids/JSON
cid_to_smiles: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/70431/property/CanonicalSMILES/JSON
991-84-4|70431|CCCCCCCCSC1=NC(=NC(=N1)NC2=CC(=C(C(=C2)C(C)(C)C)O)C(C)(C)C)SCCCCCCCC
cas_to_cid: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/88049-74-5/cids/JSON
HTTPError while requesting cas 88049-74-5
cas_to_cid: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/4161-24-4/cids/JSON
cid_to_smiles: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/20111/property/CanonicalSMILES/JSON
4161-24-4|20111|CCCCOCCCCO
cas_to_cid: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/20018-09-1/cids/JSON
cid_to_smiles: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/62738/property/CanonicalSMILES/JSON
20018-09-1|62738|CC1=CC=C(C=C1)S(=O)(=O)C(I)I
cas_to_cid: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/59-50-7/cid

BadRequestError: ignored

In [ ]:
df.head()

,"CAS,CID,SMILES"
0,"78-63-7,6545,CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C"
1,"68259-36-9,109956,CC(C)(C)CC(C)(C)C1=CC2=C(C=C..."
2,"6891-44-7,81318,CC(=C)C(=O)OCC[N+](C)(C)C.COS(..."
3,"68784-12-3,53850222,CC(C)CCCCCCCCCCCCCC=CC1CC(..."
4,"32760-80-8,15865404,CC(C)C1=CC=CC=C1.[CH-]1C=C..."


In [ ]:
!pip install pubchempy
import pubchempy as pcp
import pandas as pd

df = pd.read_csv('/content/sample_data/RESULT_Substances_with_CAS_with_SMILES3.csv', sep=",", engine='python')
df=df.drop_duplicates()


data = []

for i in df['SMILES']:
    print(i)
    try:
        props = pcp.get_properties(['MolecularFormula', 'MolecularWeight','InChI', 'InChIKey', 'IUPACName', 
                                'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 
                                'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 
                                'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 
                                'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 
                                'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 
                                'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 
                                'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 
                                'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 
                                'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 
                                'EffectiveRotorCount3D', 'ConformerCount3D'], i, 'smiles')
        data.append(props)
    except KeyError as e:
        continue

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CC(C)(C)OOC(C)(C)CCC(C)(C)OOC(C)(C)C
CC(C)(C)CC(C)(C)C1=CC2=C(C=C1)C(=CC=C2)NC3=CC=CC=C3
CC(=C)C(=O)OCC[N+](C)(C)C.COS(=O)(=O)[O-]
CC(C)CCCCCCCCCCCCCC=CC1CC(=O)OC1=O
CC(C)C1=CC=CC=C1.[CH-]1C=CC=C1.F[P-](F)(F)(F)(F)F.[Fe+2]
C1=CC=C(C=C1)C2=NC(=NC(=N2)NC3=CC=CC4=C3C(=O)C5=CC=CC=C5C4=O)NC6=CC=CC7=C6C(=O)C8=CC=CC=C8C7=O
CC(COCC(C)OC(=O)C1=CC=CC=C1)OC(=O)C2=CC=CC=C2
CC(F)F
C(CCCCCN)CCCCN
CCCCCCCCCCCCSCCC(=O)OCCOCCOCCOCCOC(=O)CCSCCCCCCCCCCCC
CCCC1C2C(C(OC(O2)C3=CC=C(C=C3)CCC)C(CO)O)OC(O1)C4=CC=C(C=C4)CCC
C=CC1CCC(C(C1)C=C)C=C
CC(C)(C)C1=CC(=CC(=C1O)C(C)(C)C)CCC(=O)NNC(=O)CCC2=CC(=C(C(=C2)C(C)(C)C)O)C(C)(C)C
C(COC(=O)CBr)OC(=O)CBr
CCO[Si](CC[Si](OCC)(OCC)OCC)(OCC)OCC
C1CCC(C(C1)C(=O)[O-])C(=O)[O-].[Ca+2]
C1CCCCCCC1
C(CC(CBr)(C#N)Br)C#N
CC1=C(C=C(C=C1)C(=C)C)C
CCCCCCC(CS(=O)(=O)O)S(=O)(=O)O
CCCCN(C1CC(N(C(C1)(C)C)C)(C)C)C2=NC(=NC(=N2)NCCCN(CCN(CCCNC3=NC(=NC(=N3)N(CCCC)C4CC(N(C(C4)(C)C)C)(C)C)N(CC

In [ ]:
ld=len(data)

In [ ]:
rows = []
columns = data[0][0].keys()

for i in range(ld):
    rows.append(data[i][0].values())

props_df = pd.DataFrame(data=rows, columns=columns) 
props_df.insert(1, 'SMILES', df['SMILES'], True)
props_df.head()
props_df.to_csv('FCN Data with Properties.csv', index=False)

In [ ]:
# OFAS Interested Data Source 5: Pubmed


# URL: https://www.ncbi.nlm.nih.gov/




# STEP 1: Install libraries


!pip install git+https://github.com/mbullmanFHCRC/pymed
!pip install biopython
import pandas as pd
props_df = pd.read_csv("/content/sample_data/FCN Data with Properties_Final.csv")

# STEP 2: Obtain set of unique IUPAC names


unique_names = props_df['IUPACName'].drop_duplicates()
unique_names=pd.DataFrame(unique_names,columns=['IUPACName'])
unique_names.IUPACName.dropna()
unique_names=unique_names.loc[unique_names['IUPACName'].str.contains("[a-zA-Z]+$",na=False)]
#unique_names=unique_names[1:20]
unique_names.to_csv('/content/sample_data/unique_names.csv')

# STEP 3: Define function to extract data from pubmed


def pubmed_query_to_df(query):
    df_list = []
    import time
    from pymed import PubMed
    import pandas as pd
    pubmed = PubMed(tool="PubMedSearcher", email="thomas.heiman@fda.hhs.gov")

## PUT YOUR SEARCH TERM HERE ##
    search_term = query
    time.sleep(3) #because I dont want to slam them with requests
#search_term = '3-hydroxy-2-(hydroxymethyl)-2-methylpropanoic '
    results = pubmed.query(search_term, max_results=500)
    articleList = []
    articleInfo = []
    for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
# We need to convert it to dictionary with available function

        articleDict = article.toDict()
        articleList.append(articleDict)
# Generate list of dict records which will hold all article details that could be fetch from 
 #PUBMED API

        for article in articleList:
#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
            pubmedId = article['pubmed_id'].partition('\n')[0]
    # Append article info to dictionary 
            try:
                articleInfo.append({u'pubmed_id':pubmedId,
                       u'title':article['title'],
                       u'keywords':article['keywords'],
                       u'meshheading':article['mesh_headings'],
                       u'journal':article['journal'],
                       u'abstract':article['abstract'],
                      # u'conclusions':article['conclusions'],
                       #u'methods':article['methods'],
                      # u'results': article['results'],
                       #u'copyrights':article['copyrights'],
                       u'doi':article['doi'],
                       u'publication_date':article['publication_date'], 
                       u'authors':article['authors']})
            except KeyError as e:
                continue
# Generate Pandas DataFrame from list of dictionaries
    articlesPD = pd.DataFrame.from_dict(articleInfo)

#Add the query to the first column

    articlesPD.insert(loc=0,  column='Query', value=search_term)
    df_list.append(articlesPD)
    data = pd.concat(df_list, axis=1)
    return data
# STEP 4: Run program on all the unique IUPAC names

df_list3 = []
num_unique=unique_names.shape[0]
query=unique_names

for ind in unique_names.index:
    query= unique_names['IUPACName'][ind]
#for i in range(num_unique):
#    query=unique_names.loc[i]
    try:
        pm_results=pubmed_query_to_df(query)
        if  pm_results.shape[0]<1:
            print("no results for", query)
        else:
            df_list3.append(pm_results)
    except KeyError as e:
                continue
data3 = pd.concat(df_list3)

# STEP 5: Save data to file
#data3.to_csv ('/content/sample_data/pubmed_data.csv', index = None, header=True)

data3.to_csv ('/content/sample_data/PM.csv', index = None, header=True)

# STEP 6: Look at extracted data

data3.head(5)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mbullmanFHCRC/pymed to /tmp/pip-req-build-up114c3o
  Running command git clone -q https://github.com/mbullmanFHCRC/pymed /tmp/pip-req-build-up114c3o
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
no results for N-phenyl-6-(2,4,4-trimethylpentan-2-yl)naphthalen-1-amine
no results for methyl sulfate;trimethyl-[2-(2-methylprop-2-enoyloxy)ethyl]azanium
no results for 3-(16-methylheptadec-1-enyl)oxolane-2,5-dione
no results for 1-[[4-[(9,10-dioxoanthracen-1-yl)amino]-6-phenyl-1,3,5-triazin-2-yl]amino]anthracene-9,10-dione
no results for 1-(2-benzoyloxypropoxy)propan-2-yl benzoate
no results for 2-[2-[2-[2-(3-dodecylsulfanylpropanoyloxy)ethoxy]ethoxy]ethoxy]ethyl 3-dodecylsulfanylpropanoate
no results for 1-[8-propyl-2,6-bis(4-propylphenyl)-4,4a,8,8a-tetrahydro-[1,3]dioxino[5,4-d][1,3]dioxin-4-yl]ethane-1,2-diol
no

,Query,pubmed_id,title,keywords,meshheading,journal,abstract,doi,publication_date,authors
0,"2,5-bis(tert-butylperoxy)-2,5-dimethylhexane",15543543,Physical characterization of coupled poly(lact...,[],"[{'Citrates': 'N'}, {'Lactic Acid': 'N'}, {'Ma...",Macromolecular bioscience,Acetyl triethyl citrate (ATC) was used as a pl...,10.1002/mabi.200400076,2004-11-16,"[{'lastname': 'Zhang', 'firstname': 'Jian-Feng..."
1,"2,5-bis(tert-butylperoxy)-2,5-dimethylhexane",15543543,Physical characterization of coupled poly(lact...,[],"[{'Citrates': 'N'}, {'Lactic Acid': 'N'}, {'Ma...",Macromolecular bioscience,Acetyl triethyl citrate (ATC) was used as a pl...,10.1002/mabi.200400076,2004-11-16,"[{'lastname': 'Zhang', 'firstname': 'Jian-Feng..."
2,"2,5-bis(tert-butylperoxy)-2,5-dimethylhexane",15244463,Mechanical properties of poly(lactic acid)/sta...,[],"[{'Chemical Phenomena': 'N'}, {'Chemistry, Phy...",Biomacromolecules,Blending poly(lactic acid) (PLA) with wheat st...,10.1021/bm0400022,2004-07-13,"[{'lastname': 'Zhang', 'firstname': 'Jian-Feng..."
0,"1,1-difluoroethane",35429657,Periarticular calcifications containing giant ...,"[""Huffing"", ALPL gene, Alkaline phosphatase, C...","[{'Alkaline Phosphatase': 'N'}, {'Bone Disease...",Bone,Inhalant use disorder is a psychiatric conditi...,10.1016/j.bone.2022.116421,2022-04-16,"[{'lastname': 'Salles Rosa Neto', 'firstname':..."
1,"1,1-difluoroethane",35429657,Periarticular calcifications containing giant ...,"[""Huffing"", ALPL gene, Alkaline phosphatase, C...","[{'Alkaline Phosphatase': 'N'}, {'Bone Disease...",Bone,Inhalant use disorder is a psychiatric conditi...,10.1016/j.bone.2022.116421,2022-04-16,"[{'lastname': 'Salles Rosa Neto', 'firstname':..."


In [ ]:
#This is the number of unique documents
len(data3['pubmed_id'].unique())

38424

In [ ]:
#This is the number of rows
data3['pubmed_id'].shape


(14075516,)

In [3]:
meta_dfu = pd.read_csv("/content/sample_data/PM_dfu.csv",sep=',', error_bad_lines=False,engine='python')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 10877: unexpected end of data


In [52]:
del meta_dfu

In [4]:
meta_dfu = meta_dfu.loc[:, ~meta_dfu.columns.str.contains('^Unnamed')]

In [56]:
meta_dfu.columns

Index(['Query', 'pubmed_id', 'title', 'keywords', 'meshheading', 'journal',
       'abstract', 'doi', 'publication_date', 'authors'],
      dtype='object')

In [5]:
FCN_Data = pd.read_csv("/content/sample_data/FCN Data with Properties_Final.csv",sep=',', error_bad_lines=False,engine='python')

In [92]:
FCN_Data.columns

Index(['CID', 'SMILES', 'MolecularFormula', 'MolecularWeight', 'InChI',
       'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 'MonoisotopicMass',
       'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount',
       'RotatableBondCount', 'HeavyAtomCount', 'IsotopeAtomCount',
       'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount',
       'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount',
       'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D',
       'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D',
       'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D',
       'FeatureCationCount3D', 'FeatureRingCount3D',
       'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D',
       'EffectiveRotorCount3D', 'ConformerCount3D'],
      dtype='object')

In [6]:
CAS_SMILES2 = pd.read_csv('/content/sample_data/RESULT_Substances_with_CAS_with_SMILES3.csv', sep=",", engine='python')

In [93]:
CAS_SMILES2.columns

Index(['CAS', 'CID', 'SMILES'], dtype='object')

In [7]:
CAS_SMILES2 = CAS_SMILES2.loc[:, ~CAS_SMILES2.columns.str.contains('^Unnamed')]
CAS_SMILES2.head(3)
CAS_SMILES2=CAS_SMILES2.drop_duplicates()
CAS_SMILES2.shape

(721, 3)

In [8]:

CAS_FCN = pd.merge(CAS_SMILES2, FCN_Data, on='SMILES', how='right')
CAS_FCN2=CAS_FCN.drop_duplicates()
CAS_FCN2.shape


(749, 41)

In [96]:
meta_dfu.columns

Index(['Query', 'pubmed_id', 'title', 'keywords', 'meshheading', 'journal',
       'abstract', 'doi', 'publication_date', 'authors'],
      dtype='object')

In [97]:
len(meta_dfu['Query'])

6452

In [98]:
len(CAS_FCN2['IUPACName'])

749

In [12]:
FCN_Pubmed=pd.merge(meta_dfu, CAS_FCN2, left_on='Query', right_on='IUPACName') 
FCN_Pubmed.shape

(12569, 51)

In [10]:
FCN_Pubmed3=FCN_Pubmed.sample(n=500, random_state=1)


In [13]:
FCN_Pubmed.to_csv ('/content/sample_data/FCN_Pubmed.csv', index = None, header=True)

In [ ]:
#Now we will try to identify the UMLS entities from the abstracts

In [2]:
!pip install -U spacy
!pip install scispacy

## Install scispaCy models
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_craft_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_jnlpba_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_lg-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.9 MB/s 
     |████████████████████████████████| 71 kB 1.3 MB/s 
     |████████████████████████████████| 13.5 MB 18.3 MB/s 
     |████████████████████████████████| 188 kB 43.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.6 MB 1.1 MB/s 
     |████████████████████████████████| 5.8 MB 3.8 MB/s 
     |████████████████████████████████| 660 kB 39.6 MB/s 
     |████████████████████████████████| 10.1 MB 41.7 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.4.0-py3-none-any.whl size=15660350 sha256=d9a11ea22cd232c5b90cb2349516a145e40b44b9522f5578babc35d9181a7204
  Stored in directory: /root/.cache/pip/wheels/31/4b/3a/03f2732eb69789a7d

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125.1 MB 46 kB/s 
  Created wheel for en-ner-craft-md: filename=en_ner_craft_md-0.4.0-py3-none-any.whl size=125689480 sha256=8c9b50246334c12b8dabda06d728252b8fdf1cd751216f146c3fbcb3334bda29
  Stored in directory: /root/.cache/pip/wheels/dd/f9/93/049ab5343fded423d6a809bcf595f4cfdd955404382fa8f085
Successfully built en-ner-craft-md
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125.1 MB 65 kB/s 
  Created wheel for en-ner-jnlpba-md: filename=en_ner_jnlpba_md-0.4.0-py3-none-any.whl size=125691806 sha256=ae6fc9fdbb92a20f25a266d0741558dd63d055b3ad78ef3ecf177e9571509989
  Stored in directory: /root/.cache/pip/wheels/ed/83/4a/2e7f532ded1476eb3750f2dec8ae8e1ebd0d890ca29c9146c7
Successfully built en-ner-jnlpba-md
Looking in indexes: https://pypi.org/simple, https://us

In [3]:
import scispacy
import spacy

#Core models
import en_core_sci_sm
import en_core_sci_lg

#NER specific models
import en_ner_craft_md
import en_ner_bc5cdr_md
import en_ner_jnlpba_md
import en_ner_bionlp13cg_md

#Tools for extracting & displaying data
from spacy import displacy
import pandas as pd

In [4]:
 #Load the models
nlp_cr = en_ner_craft_md.load()
nlp_bc = en_ner_bc5cdr_md.load()
nlp_bi = en_ner_bionlp13cg_md.load()
nlp_jn = en_ner_jnlpba_md.load()

In [5]:
def add_cr(abstractList, doiList):
    i = 0
    table= {"doi":[], "Entity":[], "Class":[]}
    for doc in nlp_cr.pipe(abstractList):
        doi = doiList[i]
        for x in doc.ents:
          table["doi"].append(doi)
          table["Entity"].append(x.text)
          table["Class"].append(x.label_)
        i +=1
    return table

In [6]:
def add_bc(abstractList, doiList):
    i = 0
    table= {"doi":[], "Entity":[], "Class":[]}
    for doc in nlp_bc.pipe(abstractList):
        doi = doiList[i]
        for x in doc.ents:
          table["doi"].append(doi)
          table["Entity"].append(x.text)
          table["Class"].append(x.label_)
        i +=1
    return table
    

In [7]:
def add_jn(abstractList, doiList):
    i = 0
    table= {"doi":[], "Entity":[], "Class":[]}
    for doc in nlp_jn.pipe(abstractList):
        doi = doiList[i]
        for x in doc.ents:
          table["doi"].append(doi)
          table["Entity"].append(x.text)
          table["Class"].append(x.label_)
        i +=1
    return table

In [8]:
def add_bi(abstractList, doiList):
    i = 0
    table= {"doi":[], "Entity":[], "Class":[]}
    for doc in nlp_bi.pipe(abstractList):
        doi = doiList[i]
        for x in doc.ents:
          table["doi"].append(doi)
          table["Entity"].append(x.text)
          table["Class"].append(x.label_)
        i +=1
    return table

In [9]:
import pandas as pd

#Read in file
meta_dfu = pd.read_csv("/content/sample_data/PM_dfu.csv",sep=',', error_bad_lines=False,engine='python')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [44]:
meta_dfu.head(2)

,Query,pubmed_id,title,keywords,meshheading,journal,abstract,doi,publication_date,authors,...,Unnamed: 4604,Unnamed: 4605,Unnamed: 4606,Unnamed: 4607,Unnamed: 4608,Unnamed: 4609,Unnamed: 4610,Unnamed: 4611,Unnamed: 4612,Unnamed: 4613
0,"2,5-bis(tert-butylperoxy)-2,5-dimethylhexane",15543543,Physical characterization of coupled poly(lact...,[],"[{'Citrates': 'N'}, {'Lactic Acid': 'N'}, {'Ma...",Macromolecular bioscience,Acetyl triethyl citrate (ATC) was used as a pl...,10.1002/mabi.200400076,11/16/2004,"[{'lastname': 'Zhang', 'firstname': 'Jian-Feng...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"2,5-bis(tert-butylperoxy)-2,5-dimethylhexane",15244463,Mechanical properties of poly(lactic acid)/sta...,[],"[{'Chemical Phenomena': 'N'}, {'Chemistry, Phy...",Biomacromolecules,Blending poly(lactic acid) (PLA) with wheat st...,10.1021/bm0400022,7/13/2004,"[{'lastname': 'Zhang', 'firstname': 'Jian-Feng...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pandas as pd

#Read in fileThis notebook assembles the data. First we extract the data for FCS (Food Contact Substances))

#Sort out blank abstracts
df = meta_dfu.dropna(subset=['abstract'])

#Create lists
doiList = df['doi'].tolist()
abstractList = df['abstract'].tolist()

#Add all entity value pairs to table (run one at a time, each ones takes ~20 min)
table_cr = add_cr(abstractList, doiList)
trans_cr_df = pd.DataFrame(table_cr)
trans_cr_df.shape

(91389, 3)

In [11]:
trans_cr_df.to_csv ("/content/sample_data/trans_cr_df.csv", index=False)

In [12]:
table_bc = add_bc(abstractList, doiList)
trans_bc_df = pd.DataFrame(table_bc)
trans_bc_df.to_csv ("/content/sample_data/trans_bc_df.csv", index=False)

In [13]:
table_bi = add_bi(abstractList, doiList)
trans_bi_df = pd.DataFrame(table_bi)
trans_bi_df.to_csv ("/content/sample_data/trans_bi_df.csv", index=False)

In [14]:
table_jn = add_jn(abstractList, doiList)
trans_jn_df = pd.DataFrame(table_jn)
trans_jn_df.to_csv ("/content/sample_data/trans_jn_df.csv", index=False)

In [16]:
trans_cr_df = pd.read_csv("/content/sample_data/trans_jn_df.csv")
trans_bc_df = pd.read_csv("/content/sample_data/trans_bc_df.csv")
trans_bi_df = pd.read_csv("/content/sample_data/trans_bi_df.csv")
trans_jn_df = pd.read_csv("/content/sample_data/trans_jn_df.csv")

trans_df =trans_cr_df.append([trans_bc_df,trans_bi_df,trans_jn_df ])
trans_df2=trans_df.drop_duplicates()
trans_df2.to_csv ("/content/sample_data/Entity_pairings.csv", index=False)
trans_df2.shape

(170922, 3)

In [20]:
trans_cr_df

,doi,Entity,Class
0,10.1002/mabi.200400076,T(g,PROTEIN
1,10.1002/mabi.200400076,T(c,PROTEIN
2,10.1002/mabi.200400076,T(m,PROTEIN
3,10.1002/mabi.200400076,PLA/starch/MA/L101,DNA
4,10.1021/bm0400022,PLA,PROTEIN
...,...,...,...
246601,10.1016/j.aninu.2022.03.004,xylanase,PROTEIN
246602,10.1016/j.aninu.2022.03.004,xylanase,PROTEIN
246603,10.1016/j.aninu.2022.03.004,cellulase,PROTEIN
246604,10.1016/j.aninu.2022.03.004,xylanase,PROTEIN
